# **Lexicon**

In [1]:
import numpy as np
import pandas as pd

### Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### DB 연결
# !pip install pymysql
import pymysql

### DB에 저장
import sqlalchemy
from sqlalchemy import create_engine

### 실시간 주식가격 데이터
# !pip install finance-datareader
import FinanceDataReader as fdr

### 텍스트 분석
## KoNLPy
# 1) JAVA 설치, 2) Python 버전과 맞는 JPype1-py3 설치, 3) !pip install konlpy, 4) 설치 경로에서 jvm.py 파일 코드 67번 줄 주석 처리 
from konlpy.tag import Okt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
## FastText
# !pip install gensim
# !pip install fasttext
import fasttext
import fasttext.util
# Facebook 한국어 Embedding 모델 다운로드 → 한 번만 설치하면 됨
fasttext.util.download_model('ko', if_exists='ignore')
# 유사도 계산
from gensim import models

### 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

### 모델 저장 및 로드
import joblib

### 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf

print('GPU', '사용 가능' if tf.config.experimental.list_physical_devices('GPU') else '사용 불가능')

GPU 사용 불가능


In [3]:
!nvidia-smi

Sat Sep 18 10:11:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 452.56       Driver Version: 452.56       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce MX450      WDDM  | 00000000:2D:00.0 Off |                  N/A |
| N/A   48C    P8    N/A /  N/A |    119MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Read KOSELF & Test Data**

### **① KOSELF 감성 어휘 사전**

In [4]:
# 블로그에서 가져온 기본적인 한국어 긍부정 텍스트 목록
with open('positive_words_self.txt', encoding='utf-8') as pos_blog:
    positive_blog = pos_blog.readlines()
positive_blog = [pos_blog.replace('\n', '') for pos_blog in positive_blog]
with open('negative_words_self.txt', encoding='utf-8') as neg_blog:
    negative_blog = neg_blog.readlines()
negative_blog = [neg_blog.replace('\n', '') for neg_blog in negative_blog]

# KOSELF 감성 어휘 사전
with open('KOSELF_pos.txt', encoding='utf-8') as pos:
    positive = pos.readlines()
positive = [pos.replace('\n', '') for pos in positive]
with open('KOSELF_neg.txt', encoding='utf-8') as neg:
    negative = neg.readlines()
negative = [neg.replace('\n', '') for neg in negative]

### **② 2018년 Hyundai Data**

In [5]:
lexicon_2018 = pd.read_csv('../../../../Code/Data/Test/Stock-Year/2018_hyundai.csv')

lexicon_2018.head(1)

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization
0,현대차,5380,매일경제,2018010100,스마트베타ETF 고공행진 새해도 이어질까,http://news.mk.co.kr/newsRead.php?year=2018&no=29,\n\n\n 수수료가 싼 상장지수펀드(ETF)에 펀드매니저가 종목을 고르는 액티브 ...,2018-01-02,0,150500,153000,149000,149500,731763,-0.041667,-1,-1,수수료 상장 지수 펀드 펀드매니저 종목 액티브 펀드 특성 가미 스마트 베타 대한 목...


In [6]:
lexicon_2018[lexicon_2018['text'].isna()]

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization


In [7]:
lexicon_2018.dropna(axis=0, inplace=True)
lexicon_2018 = lexicon_2018.reset_index(drop=True)
lexicon_2018.isna().sum()

st_n                 0
st_cd                0
news                 0
datetime             0
title                0
url                  0
text                 0
date                 0
time                 0
Open                 0
High                 0
Low                  0
Close                0
Volume               0
Change               0
UpDown               0
Extremely_Changed    0
Tokenization         0
dtype: int64

## **Calculate Cosine Similarity**

In [8]:
# 이미 학습된 FastText 내장 한국어 모델
try:
    print(ko_model, '모델 로드 불필요')
except:
    ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

In [9]:
limit_number = 15000
sys.setrecursionlimit(limit_number)

In [10]:
pos_lexicon_2018 = pd.DataFrame(columns=['date', 'news_num', 'KOSELF_pos_word', 'news_word', 'cosine_similarity', 'frequency'])   # 'frequency' 구현할 방법은 추후 생각...ㅠㅠ

# KOSELF_pos와의 Cosine Similarity 계산
for x in range(len(lexicon_2018['Tokenization'])):
    news_num = x+1
    for y in range(len(positive)):
        for z in range(len(list(set(lexicon_2018['Tokenization'][x].split())))):
            if (ko_model.wv.similarity(positive[y], list(set(lexicon_2018['Tokenization'][x].split()))[z]) >= 0.5) and (ko_model.wv.similarity(positive[y], list(set(lexicon_2018['Tokenization'][x].split()))[z]) != 1.0):
                freq = 0
                for w in range(len(lexicon_2018['Tokenization'][x].split())):
                    if lexicon_2018['Tokenization'][x].split()[w] == list(set(lexicon_2018['Tokenization'][x].split()))[z]:
                        freq += 1
                data = {
                    'date': lexicon_2018['date'][x],
                    'news_num': news_num,
                    'KOSELF_pos_word': positive[y],
                    'news_word': list(set(lexicon_2018['Tokenization'][x].split()))[z],
                    'cosine_similarity': ko_model.wv.similarity(positive[y], list(set(lexicon_2018['Tokenization'][x].split()))[z]),
                    'frequency': freq
                    }
                pos_lexicon_2018 = pos_lexicon_2018.append(data, ignore_index=True)
                print("***{0} Cosine Similarity between <{1}> & <{2}> : {3}".format(lexicon_2018['date'][x], positive[y], list(set(lexicon_2018['Tokenization'][x].split()))[z], ko_model.wv.similarity(positive[y], list(set(lexicon_2018['Tokenization'][x].split()))[z])))

***2018-01-02 Cosine Similarity between <경신> & <기록> : 0.522990882396698
***2018-01-02 Cosine Similarity between <장점> & <특성> : 0.5244326591491699
***2018-01-02 Cosine Similarity between <중요한> & <가장> : 0.5371254086494446
***2018-01-02 Cosine Similarity between <중요한> & <중요> : 0.5799874067306519
***2018-01-02 Cosine Similarity between <추진> & <중인> : 0.539746880531311
***2018-01-02 Cosine Similarity between <추진> & <추진> : 0.9999999403953552
***2018-01-02 Cosine Similarity between <추진> & <사업> : 0.5213737487792969
***2018-01-02 Cosine Similarity between <추진> & <계획> : 0.5205424427986145
***2018-01-02 Cosine Similarity between <추진> & <사업> : 0.5213737487792969
***2018-01-02 Cosine Similarity between <추진> & <진행> : 0.5358568429946899
***2018-01-02 Cosine Similarity between <적극적으로> & <적극> : 0.6639531850814819
***2018-01-02 Cosine Similarity between <추진> & <추진> : 0.9999999403953552
***2018-01-02 Cosine Similarity between <추진> & <진행> : 0.5358568429946899
***2018-01-02 Cosine Similarity between <충족> & <

In [11]:
pos_lexicon_2018

,date,news_num,KOSELF_pos_word,news_word,cosine_similarity,frequency
0,2018-01-02,1,경신,기록,0.522991,3
1,2018-01-02,1,장점,특성,0.524433,1
2,2018-01-02,2,중요한,가장,0.537125,1
3,2018-01-02,2,중요한,중요,0.579987,1
4,2018-01-02,2,추진,중인,0.539747,1
...,...,...,...,...,...,...
19090,2019-01-02,4038,추진,계획,0.520542,2
19091,2019-01-02,4038,추진,사업,0.521374,4
19092,2019-01-02,4038,해결,방안,0.531075,2
19093,2019-01-02,4038,해결,극복,0.506462,2


In [12]:
pos_lexicon_2018.to_csv('../../../../Code/Data/Test/Stock-Year/pos_hyundai_2018.csv', index=False)

In [13]:
a = list(set(list(pos_lexicon_2018['news_word'])))

a

['확립',
 '견조할',
 '웰바이오텍',
 '토픽스지수',
 '엠에스',
 '설진훈',
 '전개',
 '니혼덴산',
 '랙티브',
 '플릿물량',
 '최진명',
 '필루즈',
 '블거졌다',
 '리장시',
 '니케이지수',
 '실패',
 '단점',
 '적극',
 '조사조사',
 '말혔다',
 '임미란',
 '마련',
 '추동훈',
 '맥쿼리인프라',
 '중거리핵전력조약',
 '릴랙션',
 '중대조치',
 '리밸런싱했다',
 '중인',
 '카레이스',
 '침체',
 '계획',
 '겹악재',
 '딥글린',
 '모하비사막',
 '일장일단',
 '뉴파워트럭',
 '상쇄됐',
 '사구조',
 '중소협렵체',
 '특징',
 '확충',
 '메켄지',
 '키프트카',
 '밑돌',
 '인플렉트',
 '완판되',
 '닌빙성장',
 '변규칠',
 '애블린',
 '우호',
 '밴텍디엔씨',
 '광풍',
 '컨트럴타워',
 '매경테스트',
 '것이므',
 '횡보세',
 '딥글린트간',
 '장단점',
 '향상',
 '문제',
 '세컨카',
 '최고점',
 '얀택스',
 '컨트랙트라',
 '박봉흠',
 '주요한',
 '벤처케피탈',
 '자공시',
 '펙티브',
 '폭락',
 '리둥성',
 '고지역',
 '탭앤고',
 '라이브플래닛',
 '남경시',
 '본격',
 '화훼이',
 '헛말이었',
 '류영재',
 '요구',
 '고자세',
 '팜스코팜스코',
 '어드밴티',
 '이징도',
 '밸류액트',
 '론칭행사',
 '페이턴트리절트',
 '웃돈',
 '활황',
 '보락보락',
 '캔서롭캔서롭',
 '이유섭',
 '프로그램매매',
 '수집량',
 '차익',
 '담도굉',
 '기지원',
 '확대',
 '폐쇄되',
 '액티브펀드',
 '케이워터',
 '어닝쇼크으',
 '칼람바',
 '글라투스',
 '윤여삼',
 '뒤집혔',
 '배익철',
 '카투홈',
 '의미',
 '자양시',
 '해법',
 '앗샤르담',
 '박유악',
 '마그비',
 '훈풍',
 '롱텀펀드',
 '구현',
 '방안',
 '

In [14]:
b = list(set(list(pos_lexicon_2018[pos_lexicon_2018['cosine_similarity']>=0.7]['news_word'])))

b

['우세', '약세', '추천', '실패', '밑돈', '최고점', '단점', '추진', '불황', '앰엔씨웍스']

In [15]:
c = list(set(list(pos_lexicon_2018[pos_lexicon_2018['cosine_similarity']>=0.65]['news_word'])))

c

['감소',
 '추진',
 '밴텍디엔씨',
 '웃돌',
 '불황',
 '것이므',
 '실패',
 '밑돈',
 '최고점',
 '단점',
 '우세',
 '적극',
 '추천',
 '헛말이었',
 '프런트그릴',
 '팜스코팜스코',
 '앰엔씨웍스',
 '약세',
 '증가']